In [1]:
using LinearAlgebra
using QuantumOptics
using Plots
plotly()
using Printf 
import QuantumOptics.steadystate as steady

In [2]:
grJ=1
exJ=0
ΩL=1.0
ΩR=1.0
γ=0.05
Γ=1.0
Δ=0.5
br=0.5
myJ1system=NLevelBasis(5)  # |1> => |J=1,m=1>, |2> => |J=1,m=0>, |1> => |J=1,m=-1>, |4> => |J=0> (excited state)
                           # |5> => environment source and sink for fly-through relaxation

NLevel(N=5)

In [11]:
Bmin=-5.0
Bmax=5.0
Bstep=0.01


σ₁₄=transition(myJ1system,1,4)   # Laser excitation |1><4|
σ₃₄=transition(myJ1system,3,4)   # Laser excitation |3><4|
Γ₄₁=transition(myJ1system,4,1)   # excited state relaxation |4><1| 
Γ₄₂=transition(myJ1system,4,2)   # excited state relaxation |4><2|
Γ₄₃=transition(myJ1system,4,3)   # excited state relaxation |4><3|
γ₄₅=transition(myJ1system,4,5)   # transit relaxation (out) |4><5|
γ₃₅=transition(myJ1system,3,5)   # transit relaxation (out) |3><5|
γ₂₅=transition(myJ1system,2,5)   # transit relaxation (out) |2><5|
γ₁₅=transition(myJ1system,1,5)   # transit relaxation (out) |4><5|
γ₅₁=transition(myJ1system,5,1)   # transit relaxation (in) |5><1|
γ₅₂=transition(myJ1system,5,2)   # transit relaxation (in) |5><2|
γ₅₃=transition(myJ1system,5,3)   # transit relaxation (in) |5><3|
proj1=transition(myJ1system,1,1) # |1><1|
proj2=transition(myJ1system,2,2) # |2><2|
proj3=transition(myJ1system,3,3) # |3><4|
proj4=transition(myJ1system,4,4) # |4><4|
#id=identityoperator(proj1)
J=[sqrt(γ)*γ₄₅,
   sqrt(γ)*γ₃₅,
   sqrt(γ)*γ₂₅,
   sqrt(γ)*γ₁₅,
   sqrt(br*Γ/3)*Γ₄₁,
   sqrt(br*Γ/3)*Γ₄₂,
   sqrt(br*Γ/3)*Γ₄₃,
   sqrt(γ/3)*γ₅₁,
   sqrt(γ/3)*γ₅₂,
   sqrt(γ/3)*γ₅₃
  ]

Bvalues=[]
obs1=[]
obs2=[]
obs3=[]
obs4=[]
popEx=[]
popGr=[]
ϕ₀=(
    dm(nlevelstate(myJ1system,1))
        +dm(nlevelstate(myJ1system,2))
        +dm(nlevelstate(myJ1system,3))
        +3*dm(nlevelstate(myJ1system,5))
   )/6
for B=Bmin:Bstep:Bmax
    H=(ΩR/2)*(σ₁₄+dagger(σ₁₄)) + (ΩR/2)*(σ₃₄+dagger(σ₃₄)) + ΩL*B*proj1 - ΩL*B*proj3 - Δ*proj4
#    ϕ₀=(
#        dm(nlevelstate(myJ1system,1))
#            +dm(nlevelstate(myJ1system,2))
#            +dm(nlevelstate(myJ1system,3))
#            +3*dm(nlevelstate(myJ1system,5))
#       )/6
#    t_out, ρ_master = steady.master(H,J;rho0=ϕ₀)
#    abs=3*(Γ/ΩR)*imag(expect(σ₃₄,ρ_master[2])-expect(σ₁₄,ρ_master[2]))
#    Δϕ=3*(Γ/ΩR)*real(expect(σ₃₄,ρ_master[2])-expect(σ₁₄,ρ_master[2]))
#    rot=3*(Γ/ΩR)*real(expect(σ₁₄,ρ_master[2])+expect(σ₃₄,ρ_master[2]))
#    ell=3*(Γ/ΩR)*imag(expect(σ₁₄,ρ_master[2])+expect(σ₃₄,ρ_master[2]))
#    pop2=real(expect(proj4,ρ_master[2]))
#    pop1=real(expect(proj1,ρ_master[2])
#                +expect(proj2,ρ_master[2])
#                +expect(proj3,ρ_master[2])
#                )
    ρ_eig = steady.eigenvector(DenseOperator(H),J)
    abs=3*(Γ/ΩR)*imag(expect(σ₃₄,ρ_eig)-expect(σ₁₄,ρ_eig))
    Δϕ=3*(Γ/ΩR)*real(expect(σ₃₄,ρ_eig)-expect(σ₁₄,ρ_eig))
    rot=3*(Γ/ΩR)*real(expect(σ₁₄,ρ_eig)+expect(σ₃₄,ρ_eig))
    ell=3*(Γ/ΩR)*imag(expect(σ₁₄,ρ_eig)+expect(σ₃₄,ρ_eig))
    pop2=real(expect(proj4,ρ_eig))
    pop1=real(expect(proj1,ρ_eig)
            +expect(proj2,ρ_eig)
            +expect(proj3,ρ_eig)
            )
                #    ell=3*(Γ/ΩR)*imag(expect(σ₁₄-σ₃₄,ρ_master[2]))
#    rot=3*(Γ/ΩR)*real(expect(σ₁₄-σ₃₄,ρ_master[2]))
#    abs=3*(Γ/ΩR)*real(expect(σ₁₄+σ₃₄,ρ_master[2]))
#    Δϕ=3*(Γ/ΩR)*imag(expect(σ₁₄+σ₃₄,ρ_master[2]))

    push!(Bvalues,B)
    push!(obs1,abs)
    push!(obs2,Δϕ)
    push!(obs3,rot)
    push!(obs4,ell)
    push!(popEx,pop2)
    push!(popGr,pop1)
end

In [12]:
plot1=Plots.plot(Bvalues,obs1)
plot2=Plots.plot(Bvalues,obs2)
plot3=Plots.plot(Bvalues,popEx)
plot4=Plots.plot(Bvalues,popGr)
#plot3=Plots.plot(Bvalues,obs3)
#plot4=Plots.plot(Bvalues,obs4)
Plots.plot(plot1, plot2, plot3, plot4, layout = (2, 2), legend = false)

<!DOCTYPE html>
 
 
 Plots.jl

In [13]:
Δmin=-10.0
Δmax=10.0
Δstep=0.5
B=0.5
detunings=[]
obs1=[]
obs2=[]
obs3=[]
obs4=[]
ϕ₀=(
    dm(nlevelstate(myJ1system,1))
        +dm(nlevelstate(myJ1system,2))
        +dm(nlevelstate(myJ1system,3))
        +3*dm(nlevelstate(myJ1system,5))
   )/6
for Δ=Δmin:Δstep:Δmax
    H=(ΩR)*(σ₁₄+dagger(σ₁₄)) + (ΩR)*(σ₃₄+dagger(σ₃₄)) + ΩL*B*proj1 - ΩL*B*proj3 - Δ*proj4
#    ϕ₀=(
#        dm(nlevelstate(myJ1system,1))
#            +dm(nlevelstate(myJ1system,2))
#            +dm(nlevelstate(myJ1system,3))
#            +3*dm(nlevelstate(myJ1system,5))
#       )/6
    t_out, ρ_master = steady.master(H,J;rho0=ϕ₀)
#    abs=3*(Γ/ΩR)*imag(ρ_master[2][3,4]-ρ_master[2][1,4])
#    Δϕ=3*(Γ/ΩR)*real(ρ_master[2][3,4]-ρ_master[2][1,4])
#    rot=3*(Γ/ΩR)*real(ρ_master[2][3,4]+ρ_master[2][1,4])
#    ell=3*(Γ/ΩR)*imag(ρ_master[2][3,4]+ρ_master[2][1,4])
    abs=3*(Γ/ΩR)*imag(expect(σ₃₄,ρ_master[2])-expect(σ₁₄,ρ_master[2]))
    Δϕ=3*(Γ/ΩR)*real(expect(σ₃₄,ρ_master[2])-expect(σ₁₄,ρ_master[2]))
    rot=3*(Γ/ΩR)*real(expect(σ₁₄,ρ_master[2])+expect(σ₃₄,ρ_master[2]))
    ell=3*(Γ/ΩR)*imag(expect(σ₁₄,ρ_master[2])+expect(σ₃₄,ρ_master[2]))
    push!(detunings,Δ)
    push!(obs1,abs)
    push!(obs2,Δϕ)
    push!(obs3,rot)
    push!(obs4,ell)
end

In [14]:
plot1=Plots.plot(detunings,obs1)
plot2=Plots.plot(detunings,obs2)
plot3=Plots.plot(detunings,obs3)
plot4=Plots.plot(detunings,obs4)
Plots.plot(plot1, plot2, plot3, plot4, layout = (2, 2), legend = false)

<!DOCTYPE html>
 
 
 Plots.jl

In [15]:
p1=1
p2=1
p3=1
p4=1
myDensity=(dm(p1*nlevelstate(myJ1system,1))+dm(p2*nlevelstate(myJ1system,2))+dm(p3*nlevelstate(myJ1system,3))+dm(p4*nlevelstate(myJ1system,4)))/4
myMatrix=[1 0 0 0; 0 0 0 0; 0 0 0 0; 0 0 0 0]
myOperator=DenseOperator(myJ1system,myMatrix)
A=γ₄₃
At=dagger(A)
2*A*myDensity*At-At*A*myDensity-myDensity*At*A

LoadError: DimensionMismatch("Tried to assign data of size (4, 4) to bases of length 5 and 5!")

In [ ]:
myDensity

Operator(dim=4x4)
  basis: NLevel(N=4)
 0.25+0.0im   0.0+0.0im   0.0+0.0im   0.0+0.0im
  0.0+0.0im  0.25+0.0im   0.0+0.0im   0.0+0.0im
  0.0+0.0im   0.0+0.0im  0.25+0.0im   0.0+0.0im
  0.0+0.0im   0.0+0.0im   0.0+0.0im  0.25+0.0im

In [ ]:
dagger(σ₃₄)

Operator(dim=4x4)
  basis: NLevel(N=4)
ComplexF64[0.0-0.0im 0.0-0.0im 0.0-0.0im 0.0-0.0im; 0.0-0.0im 0.0-0.0im 0.0-0.0im 0.0-0.0im; 0.0-0.0im 0.0-0.0im 0.0-0.0im 0.0-0.0im; 0.0-0.0im 0.0-0.0im 1.0-0.0im 0.0-0.0im]

In [ ]:
    ϕ₀=(
        dm(nlevelstate(myJ1system,1))
            +dm(nlevelstate(myJ1system,2))
            +dm(nlevelstate(myJ1system,3))
            +3*dm(nlevelstate(myJ1system,5))
       )/6

Operator(dim=5x5)
  basis: NLevel(N=5)
 0.166667+0.0im       0.0+0.0im       0.0+0.0im  0.0+0.0im  0.0+0.0im
      0.0+0.0im  0.166667+0.0im       0.0+0.0im  0.0+0.0im  0.0+0.0im
      0.0+0.0im       0.0+0.0im  0.166667+0.0im  0.0+0.0im  0.0+0.0im
      0.0+0.0im       0.0+0.0im       0.0+0.0im  0.0+0.0im  0.0+0.0im
      0.0+0.0im       0.0+0.0im       0.0+0.0im  0.0+0.0im  0.5+0.0im

In [ ]:
B=1
H=(ΩR)*(σ₁₄+dagger(σ₁₄)) + (ΩR)*(σ₃₄+dagger(σ₃₄)) + ΩL*B*proj1 - ΩL*B*proj3 - Δ*proj4

Operator(dim=5x5)
  basis: NLevel(N=5)
 1.0+0.0im      ⋅           ⋅       2.0+0.0im      ⋅    
     ⋅          ⋅           ⋅           ⋅          ⋅    
     ⋅          ⋅      -1.0+0.0im   2.0+0.0im      ⋅    
 2.0+0.0im      ⋅       2.0+0.0im  -0.5+0.0im      ⋅    
     ⋅          ⋅           ⋅           ⋅          ⋅    

In [ ]:
H=(ΩR)*(σ₁₄+dagger(σ₁₄)) + (ΩR)*(σ₃₄+dagger(σ₃₄)) + ΩL*B*proj1 - ΩL*B*proj3 - Δ*proj4
    ϕ₀=(
        dm(nlevelstate(myJ1system,1))
            +dm(nlevelstate(myJ1system,2))
            +dm(nlevelstate(myJ1system,3))
            +3*dm(nlevelstate(myJ1system,5))
       )/6
t_out, ρ_master = steady.master(H,J;rho0=ϕ₀)
ρ_master[2]

Operator(dim=5x5)
  basis: NLevel(N=5)
  0.231768+1.5e-16im          0.0+0.0im  …        0.0+0.0im
       0.0+0.0im        0.0143885+0.0im           0.0+0.0im
 0.0171873+0.024396im         0.0+0.0im           0.0+0.0im
 -0.058858+0.0198414im        0.0+0.0im           0.0+0.0im
       0.0+0.0im              0.0+0.0im     0.0509908+2.0e-17im

In [ ]:
myOp=transition(myJ1system,4,1)
expect(myOp,ρ_master[2])

-0.05885802830845015 - 0.019841358713134256im